In [19]:
import networkx as nx
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import re
import datetime
import time

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
init_notebook_mode(connected=True)

import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch

import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline

import progressbar

import seaborn as sns

predtict graph theory to max profit
  by predit
  
  
  

In [284]:
######## Parameters #########

time_zone_interval = 60*8

######## Parameters #########



######## Function Start ########
def dropoff_prob_matrix(arr):
    '''compute the prbability of dropoff
    given the picuk zones
    return a matrix with row:pickup
    col: probility of dropoff'''
    total_pickups = arr.sum(axis=1)
    dim = arr.shape
    result_arr = np.zeros(dim)
    for i in range(dim[0]):
        for j in range(dim[1]):
            if total_pickups[i] == 0:
                continue
            elif arr[i,j] == 0:
                continue
            else:
                result_arr[i,j] = arr[i,j]/total_pickups[i]
    return result_arr



def Greedy_Algorithm(start_zone):
    '''Greedy algorithm search the at each states
    the minimized moving & wait time zone.
    return a path, total_trip_time & total_waste'''
    
    result_path = list() # path with zone as node
    total_trip_time = 0 # total time that are making $
    total_waste_time = 0 # total time waste in move & wait
    total_time = 24*60 # time of a day
    temp_zone = start_zone # set current zone to starting zone
    
    while total_time >= 0:
        result_path.append(temp_zone)
        # compute highest pro drop off zone
        dropoff_zone = top_dropoff_zone_t1.iloc[temp_zone][0]
        result_path.append(dropoff_zone)
        
        # compute trip_time between pickup and dropoff zone
        dropoff_trip_time =  matrix_zone_distance.iloc[temp_zone,dropoff_zone]
        total_trip_time = total_trip_time + dropoff_trip_time # add trip time to total
        total_time = total_time - dropoff_trip_time # subtract the trip time from the day
        
        # compute moving_time to other zones and the wait time in other zones
        trip_wait_time_table = pd.DataFrame(np.array(matrix_zone_distance.iloc[260])
                                            + np.array(matrix_PU_wait_time[0])).sort_values(by=0)

        # looking for the min 
        local_min_cost_zone = trip_wait_time_table.idxmin()[0]
        local_min_cost_time = trip_wait_time_table.min()[0]
        
        # update current(temp) zone, total_waste time on moving and waiting, and day
        temp_zone = local_min_cost_zone
        total_waste_time = total_waste_time + local_min_cost_time
        total_time = total_time - local_min_cost_time
        
    return result_path, total_trip_time, total_waste_time




def Randomized_Greedy_Algorithm(start_zone, randomness=3):
    '''Greedy algorithm search the at each states
    the minimized moving & wait time zone.
    return a path, total_trip_time & total_waste'''
    
    result_path = list() # path with zone as node
    total_trip_time = 0 # total time that are making $
    total_waste_time = 0 # total time waste in move & wait
    total_time = 24*60 # time of a day
    temp_zone = start_zone # set current zone to starting zone
    
    while total_time >= 0:
        result_path.append(temp_zone)
        # compute highest pro drop off zone
        dropoff_zone = top_dropoff_zone_t1.iloc[temp_zone][0]
        result_path.append(dropoff_zone)
        
        # compute trip_time between pickup and dropoff zone
        dropoff_trip_time =  matrix_zone_distance.iloc[temp_zone,dropoff_zone]
        total_trip_time = total_trip_time + dropoff_trip_time # add trip time to total
        total_time = total_time - dropoff_trip_time # subtract the trip time from the day
        
        # compute moving_time to other zones and the wait time in other zones
        trip_wait_time_table = pd.DataFrame(np.array(matrix_zone_distance.iloc[260])
                                            + np.array(matrix_PU_wait_time[0])).sort_values(by=0)

        # looking for the min 
        trip_wait_time_table = trip_wait_time_table[:20].sample(randomness)
        local_min_cost_zone = trip_wait_time_table.idxmin()[0]
        local_min_cost_time = trip_wait_time_table.min()[0]
        
        # update current(temp) zone, total_waste time on moving and waiting, and day
        temp_zone = local_min_cost_zone
        total_waste_time = total_waste_time + local_min_cost_time
        total_time = total_time - local_min_cost_time
        
    return result_path, total_trip_time, total_waste_time



class Exponential:

    def __init__(self, rate):
        self.rate = rate

    def prob_less_than_or_equal(self, t):
        rate = self.rate * t
        return 1 - np.exp(-rate)

    def prob_greater_than(self, t):
        return 1 - self.prob_less_than_or_equal(t)

    def prob_between(self, t1, t2):
        p1 = self.prob_less_than_or_equal(t1)
        p2 = self.prob_less_than_or_equal(t2)

        return p2 - p1


expo = Exponential(2)
print(expo.prob_less_than_or_equal(0.25))
print(expo.prob_between(0.25, 0.5))


######## Function End ########

0.3934693402873666
0.2386512185411911


In [34]:
%%time
yellow = pd.read_csv('./Data/yellow.csv')
yellow.head()

<string>:2: DtypeWarning:

Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.



CPU times: user 21 s, sys: 2.99 s, total: 24 s
Wall time: 24.3 s


In [35]:
# create timezone with interval 15min
yellow['Time_Diff'] = pd.to_datetime(yellow['tpep_dropoff_datetime']) - pd.to_datetime(yellow['tpep_pickup_datetime'])
yellow_new = yellow.iloc[:7781158,]
yellow_new.shape
yellow_new['Time_Diff'].mean()
yellow_new['dropoff_datetime'] = yellow_new['tpep_dropoff_datetime'].str[11:13].astype(float) * 60 + yellow_new['tpep_dropoff_datetime'].str[14:16].astype(float) + yellow_new['tpep_dropoff_datetime'].str[17:19].astype(float) / 60
yellow_new['pickup_datetime'] = yellow_new['tpep_pickup_datetime'].str[11:13].astype(float) * 60 + yellow_new['tpep_pickup_datetime'].str[14:16].astype(float) + yellow_new['tpep_pickup_datetime'].str[17:19].astype(float) / 60
yellow_new['dropoff_timezone'] = (yellow_new['dropoff_datetime'] //time_zone_interval)
yellow_new['pickup_timezone'] = (yellow_new['pickup_datetime'] // time_zone_interval)

yellow_new['pickup_date'] = pd.to_datetime(yellow_new['tpep_pickup_datetime']).dt.date
yellow_new['dropoff_date'] = pd.to_datetime(yellow_new['tpep_dropoff_datetime']).dt.date
yellow_new = yellow_new[yellow_new['pickup_date']==yellow_new['dropoff_date']]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [36]:
yellow_new.shape

(7696006, 27)

In [37]:
loc_ids = list(range(266))
num_loc_ids = len(loc_ids)

time_zones = list(yellow_new['pickup_timezone'].unique())
time_zones = sorted(time_zones)
num_time_zones = len(time_zones)
print(num_loc_ids)
print(num_time_zones)

266
3


In [38]:
## create 2-d matrix to with each row as PULocationID and each col as the time_zone
matrix_PU_time = pd.DataFrame() # create empty dataframe

for time_zone in  time_zones:
    temp_df = (time_zone_interval / (yellow_new[yellow_new['pickup_timezone']==time_zone]
                                    .groupby(['PULocationID'])['DOLocationID']
                                    .agg(['count'])))
    temp_df.rename(index=str, columns={"count": time_zone},inplace=True)
    matrix_PU_time = matrix_PU_time.merge(temp_df,
                                                how='outer',
                                                left_index = True,
                                                right_index = True)
    
matrix_PU_time.shape

(262, 3)

In [84]:
matrix_PU_time.fillna(value=time_zone_interval, inplace=True)
matrix_PU_wait_time = matrix_PU_time.set_index(matrix_PU_time.index.astype(int)).sort_index()

In [49]:
arr_time_PU_DO = np.zeros([num_time_zones,num_loc_ids,num_loc_ids])
print(('1d: each time_zone'
      +'\n2d: each pick up zone'
      +'\n3d: each dropoff zone'))
arr_time_PU_DO.shape

1d: each time_zone
2d: each pick up zone
3d: each dropoff zone


(3, 266, 266)

In [50]:
%%time
bar = progressbar.ProgressBar(max_value=num_time_zones-1)
for time_zone in range(num_time_zones):
    
    bar.update(time_zone)# print out the progress and running time
    
    temp_PU_DO = (yellow_new[yellow_new['pickup_timezone']==time_zone]
               .groupby(['PULocationID'])['DOLocationID']
               .value_counts()).unstack(fill_value=0)

    temp_arr = np.zeros([num_loc_ids,num_loc_ids])
    for i in range(num_loc_ids): 
        for j in range(num_loc_ids):
            try:
                temp_arr[i,j] = temp_PU_DO.loc[i,j]
            except:
                continue
    arr_time_PU_DO[time_zone] =  temp_arr

100% (2 of 2) |##########################| Elapsed Time: 0:00:04 ETA:  00:00:00

CPU times: user 6.84 s, sys: 936 ms, total: 7.77 s
Wall time: 7.78 s


In [51]:
arr_time_PU_DO.shape

(3, 266, 266)

In [52]:
## create a probability array that given a time zone,  it have row as a pickup loc and col as prob(dropoff loc)
arr_prob_PU_DO = np.zeros(arr_time_PU_DO.shape)  # create a 3-d array of the same shape of arr_time_PU_DO
for time_zone in range(num_time_zones):
    arr_prob_PU_DO[time_zone] = dropoff_prob_matrix(arr_time_PU_DO[time_zone])

In [54]:
## loading the distance matrix(compute from the full_yellow.csv)
M = 1000000000000 # large number
zone_dis = pd.read_csv('./Data/zone_distance_matrix.csv', index_col=0)

In [230]:
matrix_zone_distance = pd.read_csv('./Data/full_zone_distance_matrix.csv',index_col=0)
matrix_zone_distance.index = matrix_zone_distance.index + 1
temp_index = matrix_zone_distance.index
matrix_zone_distance = matrix_zone_distance.T
matrix_zone_distance.index = temp_index
matrix_zone_distance
matrix_zone_distance.shape

(265, 265)

In [231]:
ind_dis = set(matrix_zone_distance.iloc[146].index)
ind_wait_time = set(matrix_PU_wait_time[0].index)
redundant_zones = list(ind_dis.difference(ind_wait_time))
redundant_zones
matrix_zone_distance = matrix_zone_distance.drop(redundant_zones, axis=1).drop(list(map(int, redundant_zones)))
matrix_zone_distance.shape

(262, 262)

In [232]:
matrix_zone_distance
matrix_PU_wait_time
arr_prob_PU_DO.shape

(3, 266, 266)

In [234]:
matrix_prob_PU_DO_t0 = pd.DataFrame(arr_prob_PU_DO[0])
matrix_prob_PU_DO_t1 = pd.DataFrame(arr_prob_PU_DO[1])
matrix_prob_PU_DO_t2 = pd.DataFrame(arr_prob_PU_DO[2])

In [233]:
matrix_prob_PU_DO_t0.iloc[104]

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
5      0.0
6      0.0
7      0.0
8      0.0
9      0.0
10     0.0
11     0.0
12     0.0
13     0.0
14     0.0
15     0.0
16     0.0
17     0.0
18     0.0
19     0.0
20     0.0
21     0.0
22     0.0
23     0.0
24     0.0
25     0.0
26     0.0
27     0.0
28     0.0
29     0.0
      ... 
236    0.0
237    0.0
238    0.0
239    0.0
240    0.0
241    0.0
242    0.0
243    0.0
244    0.0
245    0.0
246    0.0
247    0.0
248    0.0
249    0.0
250    0.0
251    0.0
252    0.0
253    0.0
254    0.0
255    0.0
256    0.0
257    0.0
258    0.0
259    0.0
260    0.0
261    0.0
262    0.0
263    0.0
264    0.0
265    0.0
Name: 104, Length: 266, dtype: float64

In [237]:
matrix_zone_distance.iloc[104,0]

35.38333333333335

In [250]:

pd.DataFrame(np.array(matrix_zone_distance.iloc[260]) + np.array(matrix_PU_wait_time[0])).sort_values(by=0).min()


0    1.671495
dtype: float64

In [248]:
top_dropoff_zone_t1 = pd.DataFrame(matrix_prob_PU_DO_t1.idxmax(axis=1))
top_dropoff_zone_t1.iloc[98][0]

98

In [258]:
trip_wait_time_table.min()[0]

1.6714953665986592

In [292]:
path = Randomized_Greedy_Algorithm(30,randomness=5)[0]

In [227]:
freq = pd.DataFrame(arr_time_PU_DO.sum(axis=0))
mask = (freq.index.get_level_values(0)==1) 

freq[(freq.index.get_level_values(0)==1) ]

data = [go.Bar(y=freq[i],
              name=f'pickup_zone{i}')
       for i in range(100,109)]  

layout = go.Layout(title=('Drop of freqency '
                          +'of pickup zone'))
fig = go.Figure(data, layout)
iplot(fig)